In [1]:
from imp import reload
import tensorflow as tf
import numpy as np

import importlib

from source.Model.Transformer_model import Transformer



/tmp/ipykernel_134033/2078188667.py:1: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
2022-10-06 14:46:21.938638: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 14:46:21.938665: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
input_embedding = [[
  "Salut", "comment", "ca", "va", "?"
]]


output_embedding = [[
    "<START>", "Hi", "how", "are", "you", "?"
]]

def get_vocabulary(sequences):

  token_to_info = {}

  for sequence in sequences:
    for word in sequence:
      if word not in token_to_info:
        token_to_info[word] = len(token_to_info)
  
  return token_to_info

input_voc = get_vocabulary(input_embedding)
output_voc = get_vocabulary(output_embedding)

input_voc["<START>"] = len(input_voc)
input_voc["<END>"] = len(input_voc)
input_voc["<PAD>"] = len(input_voc)

output_voc["<END>"] = len(output_voc)
output_voc["<PAD>"] = len(output_voc)

print(input_voc)
print(output_voc)

def sequences_to_int(sequences, voc):
  for sequence in sequences:
    print(sequence)
    for s, word in enumerate(sequence):
      sequence[s] = voc[word]
  return np.array(sequences)

input_seq = sequences_to_int(input_embedding, input_voc)
output_seq = sequences_to_int(output_embedding, output_voc)

print("input_seq", input_seq)
print("output_seq", output_seq)

{'Salut': 0, 'comment': 1, 'ca': 2, 'va': 3, '?': 4, '<START>': 5, '<END>': 6, '<PAD>': 7}
{'<START>': 0, 'Hi': 1, 'how': 2, 'are': 3, 'you': 4, '?': 5, '<END>': 6, '<PAD>': 7}
['Salut', 'comment', 'ca', 'va', '?']
['<START>', 'Hi', 'how', 'are', 'you', '?']
input_seq [[0 1 2 3 4]]
output_seq [[0 1 2 3 4 5]]


In [3]:
transformer = Transformer(coding_block=6, dim=256, nbr_heads=8)
output = transformer((input_seq, output_seq))

2022-10-06 14:46:25.410621: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 14:46:25.410923: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-06 14:46:25.410967: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-10-06 14:46:25.410998: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-10-06 14:46:25.411029: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 5)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 embedding__layer (Embedding_La  (None, 5, 256)      1280        ['input_1[0][0]']                
 yer)                                                                                             
                                                                                                  
 embedding__layer_1 (Embedding_  (None, 6, 256)      1536        ['input_2[0][0]']            

In [4]:
output

<tf.Tensor: shape=(1, 6, 8), dtype=float32, numpy=
array([[[0.11741469, 0.12957208, 0.1276873 , 0.12777339, 0.12805915,
         0.12938951, 0.11847531, 0.12162853],
        [0.13019527, 0.13076097, 0.12291659, 0.12526767, 0.1268087 ,
         0.12538418, 0.12080977, 0.11785687],
        [0.12586147, 0.12571369, 0.12718467, 0.12199046, 0.11904333,
         0.13112497, 0.12736341, 0.12171794],
        [0.13571785, 0.12616812, 0.1174528 , 0.121117  , 0.12787803,
         0.12020996, 0.1187636 , 0.13269262],
        [0.13112095, 0.12101729, 0.12059278, 0.12510535, 0.1238372 ,
         0.12342832, 0.12979028, 0.12510777],
        [0.1257674 , 0.12481982, 0.12139805, 0.12771165, 0.12601797,
         0.1241264 , 0.12631916, 0.12383961]]], dtype=float32)>